<a href="https://colab.research.google.com/github/rizwan08-bit/rizwan08/blob/main/work_on_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


In [6]:
from google.colab import files
uploaded = files.upload()


Saving science_questions_answers.txt to science_questions_answers.txt


In [7]:
with open("science_questions_answers.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()


In [8]:
questions = []
answers = []
for i in range(len(lines)):
    if lines[i].startswith("Q:"):
        questions.append(lines[i].replace("Q:", "").strip())
    elif lines[i].startswith("A:"):
        answers.append(lines[i].replace("A:", "").strip())

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
vocab_size = len(tokenizer.word_index) + 1

In [10]:
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

In [11]:
max_length = max(max(len(seq) for seq in question_sequences), max(len(seq) for seq in answer_sequences))
question_padded = pad_sequences(question_sequences, maxlen=max_length, padding='post')
answer_padded = pad_sequences(answer_sequences, maxlen=max_length, padding='post')

In [12]:
answers_output = np.array([seq[-1] for seq in answer_padded])

In [13]:
# Define the RNN model
model = Sequential([
    Embedding(vocab_size, 64, input_length=max_length),
    SimpleRNN(128, return_sequences=True),
    SimpleRNN(64),
    Dense(64, activation='relu'),
    Dense(vocab_size, activation='softmax')  # Output layer with softmax activation
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(question_padded, answers_output, epochs=5, batch_size=4, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6152 - loss: 4.5987
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.1987
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0032
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0013


In [14]:
def chat_response(text):
    seq = tokenizer.texts_to_sequences([text])
    seq_padded = pad_sequences(seq, maxlen=max_length, padding='post')
    pred = model.predict(seq_padded, verbose=0)
    predicted_index = np.argmax(pred, axis=1)[0]  # Get the highest probability word index
    response_word = tokenizer.index_word.get(predicted_index, "Sorry, I don't understand.")
    return response_word

print(chat_response("hello"))

Sorry, I don't understand.
